# Imports

## Modules

In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import Counter
import re
from ast import literal_eval


# Scaling
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
!pip install plotly.express
import plotly.express as px

## Data

In [2]:
csv_path = '../raw_data/recipes_with_allergens.csv' 

In [3]:
df = pd.read_csv(csv_path)

In [4]:
df.shape

(4805, 54)

In [7]:
df

,Unnamed: 0,Title,Instructions,Image_Name,Ingredients,Cleaned_Ingredients,Relabelled_Ingredients,Tokenized_Ingredients,apple,banana,...,soy,walnut,almond,hazelnut,pecan,cashew,pistachio,wheat,vegetarian,vegan
0,0,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","['egg white', 'potato', 'salt', 'black pepper'...","['egg', 'potato', 'salt', 'black pepper', 'ros...","['egg', 'potato', 'salt', 'black', 'pepper', '...",0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","['milk', 'milk', 'garlic powder', 'onion powde...","['milk', 'milk', 'garlic powder', 'onion', 'sm...","['milk', 'milk', 'garlic', 'powder', 'onion', ...",0,0,...,0,0,0,0,0,0,0,0,1,1
2,2,Warm Comfort,Place 2 chamomile tea bags in a heatsafe vesse...,warm-comfort-tequila-chamomile-toddy,"['2 chamomile tea bags', '1½ oz. reposado tequ...","['chamomile tea bag', 'reposado tequila', 'lem...","['chamomile tea bag', 'reposado tequila', 'lem...","['chamomile', 'tea', 'bag', 'reposado', 'tequi...",0,0,...,0,0,0,0,0,0,0,0,1,1
3,3,Apples and Oranges,"Add 3 oz. Grand Marnier, 1 oz. Amaro Averna, a...",apples-and-oranges-spiked-cider,"['3 oz. Grand Marnier', '1 oz. Amaro Averna', ...","['grand marnier', 'amaro averna', 'pat butter'...","['grand marnier', 'amaro averna', 'butter', 'h...","['grand', 'marnier', 'amaro', 'averna', 'butte...",0,0,...,0,0,0,0,0,0,0,0,1,1
4,4,Turmeric Hot Toddy,"For the turmeric syrup, combine ½ cup hot wate...",turmeric-hot-toddy-claire-sprouse,"['¼ cup granulated sugar', '¾ tsp. ground turm...","['granulated sugar', 'turmeric', 'amontillado ...","['sugar', 'turmeric', 'amontillado sherry', 'b...","['sugar', 'turmeric', 'amontillado', 'sherry',...",0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,4800,Cornmeal Pancakes with Honey-Pecan Butter,"Using electric mixer, beat 1/2 cup butter, hon...",cornmeal-pancakes-with-honey-pecan-butter-108554,['1/2 cup (1 stick) unsalted European-style bu...,"['stick europeanstyle butter', 'honey', 'gener...","['butter', 'honey', 'generous pinch cinnamon',...","['butter', 'honey', 'generous', 'pinch', 'cinn...",0,0,...,0,0,0,0,1,0,0,0,1,0
4801,4801,Ginger-Pecan Roulade with Honey-Glazed Pecans,Preheat oven to 350°F. Line bottom and sides o...,ginger-pecan-roulade-with-honey-glazed-pecans-...,"['1/2 stick (1/4 cup) unsalted butter, melted,...","['stick butter additional brushing pan', 'peca...","['butter', 'pecan toasted cooled', 'flour', 'u...","['butter', 'pecan', 'toasted', 'cooled', 'flou...",0,0,...,0,0,0,0,1,0,0,0,1,0
4802,4802,Brownie Pudding Cake,Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten...","['allpurpose flour', 'unsweetened cocoa powder...","['flour', 'unsweetened cocoa powder', 'doublea...","['flour', 'unsweetened', 'cocoa', 'powder', 'd...",0,0,...,0,1,0,0,0,0,0,0,1,0
4803,4803,Rice with Soy-Glazed Bonito Flakes and Sesame ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...,['leftover katsuo bushi dried bonito making da...,['leftover katsuo bushi dried bonito making da...,"['leftover', 'katsuo', 'bushi', 'dried', 'boni...",0,0,...,1,0,0,0,0,0,0,0,1,1


# Filtering the dataset based on user preferences

In [8]:
df.columns

Index(['Unnamed: 0', 'Title', 'Instructions', 'Image_Name', 'Ingredients',
       'Cleaned_Ingredients', 'Relabelled_Ingredients',
       'Tokenized_Ingredients', 'apple', 'banana', 'beef', 'blueberry',
       'bread', 'butter', 'carrot', 'cheese', 'chicken', 'chocolate', 'corn',
       'egg', 'flour', 'bean', 'ham', 'cream', 'lime', 'milk', 'mushroom',
       'onion', 'potato', 'shrimp', 'spinach', 'strawberry', 'sugar', 'tomato',
       'Remove', 'Prep Time', 'Prep Time Range', 'number_of_ingredients',
       'length_of_instructions', 'complexity', 'complexity_label', 'eggs',
       'mustard', 'peanut', 'soy', 'walnut', 'almond', 'hazelnut', 'pecan',
       'cashew', 'pistachio', 'wheat', 'vegetarian', 'vegan'],
      dtype='object')

In [24]:
filters = ['milk','egg', 'Prep Time Range', 'number_of_ingredients',
       'length_of_instructions', 'complexity', 'complexity_label',
       'mustard', 'peanut', 'soy', 'walnut', 'almond', 'hazelnut', 'pecan',
       'cashew', 'pistachio', 'wheat', 'vegetarian', 'vegan']

## First filter - Vegatarien/Vegan

In [36]:
def vegfilter(df, vegetarian=False, vegan=False):
    if vegan == True:
        df_filtered = df[df['vegan']==1]
    elif vegetarian == True:
        df_filtered = df[df['vegetarian']==1]
    else:
        df_filtered = df
    return df_filtered

## Second Filter - Prep Time and Complexity

In [37]:
def difficulty(df, max_prep_time=10000, max_complexity=0):
    df_filtered = df[df['Prep Time']<=max_prep_time]
    df_filtered = df_filtered[df_filtered['complexity']<=max_complexity]
    return df_filtered

## Third Filter - Allegerns